spring boot starter project
spring boot starter parent
	<parent>
		<groupId>org.springframework.boot</groupId>
		<artifactId>spring-boot-starter-parent</artifactId>
		<version>3.2.6</version>
		<relativePath/> <!-- lookup parent from repository -->
	</parent>
    
dependency version management, inherit version from parent

- Spring Boot Starter WEB
```
<dependencies>
    <dependency>
    	<groupId>org.springframework.boot</groupId>
        <artifactId>spring-boot-starter-web</artifactId>
    </dependency>
</dependencies>
```

StarterClass, run method
```
@ComponentScan
@EnableAutoConfiguration // will automatically configure webserver(ServletWebServerFactory)by intitialize Root WebApplicationContet
public class StarterApp {

    public static main(String[] args) {
        SpringApplication springApplication = new SpringApplication(StarterApp.class); // passing in StarterApp telling Spring boot that this is the entry point of the application, the source our the application
        springApplication.run(); 
    }
}
```

run method
```
    SpringApplication.run(StarterApp.class, args);
```

```
        SpringApplication springApplication = new SpringApplication(StarterApp.class); 
        ConfigurableApplicationContext container = springApplication.run();

        YourBean yourBean = container.getBean(YourBean.class);

        yourBean.someMethod();
```

@SpringBootConfiguration: tell spring boot that this is a configuration file

@SpringBootApplication equivallent
- @SpringBootConfiguration
- @ComponentScan
- @EnableAutoConfiguration

JAR vs WAR vs EAR

SpringBoot JAR vs normal JAR vs FAT JAR

`./mvnw clean install` or `./mvnw clean package` creat a FAT JAR

FAT JAR: Is a JAR that has an embedded web server from `spring-boot-starter-web` dependency

add the maven plugin in pom.xml
```
	<build>
		<plugins>
			<plugin>
				<groupId>org.springframework.boot</groupId>
				<artifactId>spring-boot-maven-plugin</artifactId>
			</plugin>
		</plugins>
	</build>
```

you an start your application as well as run your embedded webserver with
```
java -jar yourfatjar.jar
```

|jar|fatjar|
|--|--|
|smaller eg 4kb|larger eg 19mb, has manifest|
|no dependencies|has dependencies|


manifest file in the MANIFEST.MF file
in normal jar, there is no main method define, which class will start the app?

in the manifest for fat jar it contain
- Main-Class
- Start-Class

contain jar launcher, which will spin up your application


spring boot autoconfiguration

@EnableAutoConfiguration: all the autoconfig file will be triggered, but not all beans will be created
selective bean creation using @Condition

@ConditionalOnClass(SomeClass.class)
@ConditionalOnBean("beanname")

# JavaMailSender
`JavaMailSender` is a Spring interface that simplifies sending emails in a Spring application. It provides a higher-level API for sending emails compared to using the JavaMail API directly.

Key Features
1. SimpleMailMessage Support: Send plain text emails using `SimpleMailMessage`.
2. MIME Messages: Send more complex emails with attachments, HTML content, etc., using `MimeMessage`.
3. Configuration: Easily configured via Spring's application properties.

## Set Up
1. Add Dependencies<br>
add the following to your `pom.xml`.
```
<dependency>
    <groupId>org.springframework.boot</groupId>
    <artifactId>spring-boot-starter-mail</artifactId>
</dependency>
```

2. Configuring Mail Properties
```
@Configuration
public class ClientConfiguration {

    @Bean
    JavaMailSender getMailSender() {
        JavaMailSenderImpl javaMailSenderImpl = new JavaMailSenderImpl();
        javaMailSenderImpl.setHost("smtp.gmail.com");
        javaMailSenderImpl.setUsername({username});
        javaMailSenderImpl.setPassword({password});
        javaMailSenderImpl.setPort(587);

        Properties mailProperties = new Properties();
        mailProperties.put("mail.smtp.starttls.enable", "true");
        mailProperties.put("mail.smtp.ssl.trust", "smtp.gmail.com");

        javaMailSenderImpl.setJavaMailProperties(mailProperties);
        return javaMailSenderImpl;
    }

    @Bean
    EmailService getEmailService(JavaMailSender mailSender) {
        return new EmailService(mailSender);
    }
}
```

## Sending Plaintext emails using SimpleMailMessage
```
@Service
public class EmailService {

    public JavaMailSender mailSender;

    @Autowired
    public EmailService(JavaMailSender mailSender) {
        this.mailsender = mailSender;
    }

    public void sendEmailMessage(String to, String subject, String text) {
        SimpleMailMessage message = new SimpleMailMessage();
        message.setTo(to);
        message.setSubject(subject);
        message.setText(text);
        emailSender.send(message);
    }
}
```

## MIME MEssages
MIME(Multipurpose Internet Mail Extensions) messages allow you to send complex emails that include HTML content, attachments, inline images, and more.
```
@Service
public class EmailService {

    public JavaMailSender mailSender;

    @Autowired
    public EmailService(JavaMailSender mailSender) {
        this.mailsender = mailSender;
    }

    public void sendMimeMessage(String to, String subject, String text, boolean isHtmlContent, String pathToAttachment) throws MessagingException {
        MimeMessage message = mailSender.CreateMimeMessage();

        MimeMessageHelper helper = new MimeMessageHelper(message, true);
        helper.setTo(to);
        helper.setSubject(subject);
        helper.setText(text, isHtmlContent);

        if (pathToAttachment != null) {
            File file = new File(pathToAttachment);
            if (file.exists()) {
                helper.addAttachment(file.getName(), file);
            }
        }

        mailSender.send(message);
    }
}
```

make sure,  google less secure app, enabled